In [1]:
import matplotlib
import matplotlib.pyplot as plt
import cv2
import numpy as np
import scipy as sp
from numpy.fft import fft2, ifft2, ifftshift

from scipy.stats import multivariate_normal
from scipy.sparse import spdiags
from scipy.sparse.linalg import spsolve
import scipy.ndimage.measurements as meas
import csv

#%matplotlib qt
cv2.namedWindow('spatial domain', cv2.WINDOW_NORMAL)
cv2.namedWindow('convolution', cv2.WINDOW_NORMAL)

In [2]:
#locaton of VOT 2013 dataset
path = 'vot2013/'

In [3]:
def read_frame(i):
    images = list()
    img = cv2.imread(path + str(i).zfill(3)+'.bmp', 1)
    img = process_frame(img)
    return img

def sequence(obj_id):
    f = open(path + 'list.txt', 'r')
    object_list = f.read().splitlines()
    f.close()

    f = open(path + object_list[obj_id] + '/groundtruth.txt', 'rb')
    gt = f.read().splitlines()
    f.close()
    gt = [np.array(i.split(',')).astype(np.float) for i in gt]

    images = list()
    for i in range(1, len(gt)+1):
        img = cv2.imread(path + object_list[obj_id]+ '/' + str(i).zfill(8)+'.jpg', 1)
        img = process_frame(img)
        images.append(img)
    
    return gt, images


In [4]:
def process_frame(frame):
    if frame.ndim < 3:
        frame = frame.reshape((frame.shape[0], frame.shape[1], 1))
    k=frame.shape[-1]
    return frame.astype(np.float32) / 255.

def cut(img, box):
    left, top, width, height = [int(round(i)) for i in box]
    lpad, tpad = left, top
    if lpad < 0:
        lpad = abs(lpad)
        left+=lpad
    else:
        lpad = 0
    if tpad < 0:
        tpad = abs(tpad)
        top += tpad
    else:
        tpad = 0
    if lpad > 0 or tpad > 0:
        padded = np.zeros((img.shape[0]+tpad, img.shape[1]+lpad, img.shape[2]), dtype=np.uint8)
        padded[tpad:, lpad:, :] = img[:,:,:]    
        return padded[top:top+height, left:left+width] 
    else:
        return img[top:top+height, left:left+width]

def gaussian_peak(size, mu = [0, 0], var = [[2, 0], [0, 2]]):
    x = np.linspace(-size/2,size/2,size)
    y = np.linspace(-size/2,size/2,size)
    X, Y = np.meshgrid(x,y)
    pos = np.empty(X.shape + (2,))
    pos[:, :, 0] = X; pos[:, :, 1] = Y
    rv = multivariate_normal(mu, var)
    out = np.zeros((size, size))
    out = cv2.normalize(rv.pdf(pos), out, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    return out

def fast2dconv2(x, y):
    if x.ndim < 3:
        x = x.reshape(x.shape[0], x.shape[1], 1)
    if y.ndim < 3:
        y = y.reshape(y.shape[0], y.shape[1], 1)
    h, w, k = x.shape   
    fft_y = fft2(y, axes=[0, 1], norm='ortho')
    fft_x = fft2(x, axes=[0, 1], norm='ortho')
    c = 0
    for i in range(k):
        c += fft_x[:,:,i] * fft_y[:,:,i]    
    return np.real(ifftshift(ifft2(c, axes=[0, 1], norm='ortho'), axes=(0, 1)))

In [5]:
def increment_mccf(A, B, X, y, nu=0.125, l=0.01):
    x = X[0]
    if x.ndim < 3:
        x = x.reshape(x.shape[0], x.shape[1], 1)
    hx, wx, k = x.shape
    ext_d = hx * wx
    fft_y = fft2(y, norm='ortho')
    diag_fft_y = fft_y.ravel()
    
    sXY = sXX = 0
    for x in X:
        x = x.swapaxes(-1, 0)
        x = x.swapaxes(1, 2)
        fft_x = fft2(x, norm='ortho')
        diag_fft_x = spdiags(fft_x.reshape((k, -1)),
                                 -np.arange(0, k) * ext_d, ext_d * k, ext_d).T
        sXY += diag_fft_x.conj().T.dot(diag_fft_y)
        sXX += diag_fft_x.conj().T.dot(diag_fft_x)
    
    sXY = (1 - nu) * A + nu * sXY
    sXX = (1 - nu) * B + nu * sXX
    fft_h = spsolve(sXX + l * sp.sparse.eye(sXX.shape[-1]), sXY)
    fft_h = fft_h.reshape((k, hx, wx))
    h = np.real(ifftshift(ifft2(fft_h, norm='ortho'), axes=(-2, -1)))
    h = h.swapaxes(1, 2)
    h = h.swapaxes(-1, 0)
    return h, sXY, sXX

def learn_mccf(X, y, l=0.1):
    x = X[0]
    if x.ndim < 3:
        x = x.reshape(x.shape[0], x.shape[1], 1)
    hx, wx, k = x.shape
    ext_d = hx * wx
    fft_y = fft2(y, norm='ortho')
    diag_fft_y = fft_y.ravel()
    
    sXY = sXX = 0
    for x in X:
        x = x.swapaxes(-1, 0)
        x = x.swapaxes(1, 2)
    
        fft_x= fft2(x, norm='ortho')
        diag_fft_x = spdiags(fft_x.reshape((k, -1)),
                             -np.arange(0, k) * ext_d, ext_d * k, ext_d).T
        sXY += diag_fft_x.conj().T.dot(diag_fft_y)
        sXX += diag_fft_x.conj().T.dot(diag_fft_x)
    
    fft_h = spsolve(sXX + l * sp.sparse.eye(sXX.shape[-1]), sXY)
    fft_h = fft_h.reshape((k, hx, wx))
    h = np.real(ifftshift(ifft2(fft_h, norm='ortho'), axes=(-2, -1)))
    h = h.swapaxes(1, 2)
    h = h.swapaxes(-1, 0)
    return h, sXY, sXX

def frame_score(r1, r2):
    l1, t1, w1, h1 = r1
    l2, t2, w2, h2 = r2
    l, t = max(l1, l2), max(t1, t2)
    r, b = min(l1+w1, l2+w2), min(t1+h1, t2+h2)
    score = (r-l)*(b-t) / (w1*h1 + w2*h2 - 1.*(r-l)*(b-t))
    if l > r and t > b:
        score, (l, t, -r+l, -b+t)
        #return 0, (l, t), (r, b)
    else:
        return score, (l, t, r-l, b-t)

In [6]:
def cv2rect(rect, bbox, col):
    gtbbox = [int(round(i)) for i in bbox]
    cv2.rectangle(rect, (gtbbox[0], gtbbox[1]), (gtbbox[0] + gtbbox[2], gtbbox[1] + gtbbox[3]), col, 1) 

def show(img, bbox, gt, intersec):
    cv2rect(img, gt, (255, 0, 0))
    cv2rect(img, intersec, (0,0,255))
    cv2rect(img, bbox, (0,255,0))
    cv2.imshow('spatial domain', img[:,:,:3])

    cv2.waitKey(1)     

def correct_box(bbox, dtop, dleft, scale, exp):
    left, top, width, height = bbox
    h = exp*scale*height
    w = exp*scale*width
    sheight = height + 2*h
    swidth = width + 2*w
    dtop /= hsize / sheight
    dleft /= hsize / swidth
    swidth, sheight = width*scale, height*scale
    dwidth, dheight = swidth-width, sheight-height
    left += dleft - dwidth/2.
    top += dtop - dheight/2. 
    bbox = left, top, swidth, sheight
    return bbox

def hog_features(hog, img):
    img = cv2.resize(img, (fsize, fsize))

    n_cells = (img.shape[0] // cell_size[0], img.shape[1] // cell_size[1])
    hog_feats = hog.compute(img).reshape(n_cells[1] - block_size[1] + 1,
                            n_cells[0] - block_size[0] + 1,
                            block_size[0], block_size[1], nbins) \
                   .transpose((1, 0, 2, 3, 4))  # index blocks by rows first
    # hog_feats now contains the gradient amplitudes for each direction,
    # for each cell of its group for each group. Indexing is by rows then columns.

    gradients = np.zeros((n_cells[0], n_cells[1], nbins))

    # count cells (border cells appear less often across overlapping groups)
    cell_count = np.full((n_cells[0], n_cells[1], 1), 0, dtype=int)

    for off_y in range(block_size[0]):
        for off_x in range(block_size[1]):
            gradients[off_y:n_cells[0] - block_size[0] + off_y + 1,
                      off_x:n_cells[1] - block_size[1] + off_x + 1] += \
                        hog_feats[:, :, off_y, off_x, :]


            cell_count[off_y:n_cells[0] - block_size[0] + off_y + 1,
                       off_x:n_cells[1] - block_size[1] + off_x + 1] += 1

    # Average gradients
    gradients /= cell_count
    return gradients

def process_target(img, bbox, exp, add_window):
    left, top, width, height = bbox
    h, w = exp * height, exp * width
    target = cut(img, (left-w, top-h, width+2*w, height+2*h))
    target = cv2.resize(target, (fsize, fsize))
    
    if add_window:
        for i in range(target.shape[-1]):
            target[:,:,i]
    
    target = cv2.convertScaleAbs(target)
    target = hog_features(hog_desc, target)
    return target

    
def new_pos(htarget, ker, psr_r, is_show):
    c = fast2dconv2(htarget, ker)
    it, il = np.array(np.unravel_index(c.argmax(), c.shape))
    peak = c[it, il]
    yy = c.copy()
    full_sum =  c.sum()
    c[c < peak/2.] = 0.
    a, b = it, il
    n = hsize
    r = psr_r
    y,x = np.ogrid[-a:n-a, -b:n-b]
    
    mask = x*x + y*y <= r*r
    array = np.zeros((n, n))
    array[mask] = 1.
    
    c *= array
    it, il = meas.center_of_mass(c)
    dt, dl = it - hsize/2, il - hsize/2
    peak_sum = c.sum()
    
    psr = peak_sum / full_sum
    
    if is_show:
        img = np.zeros((yy.shape[0], yy.shape[1], 3))
        img[:,:,0] = img[:,:,1] = img[:,:,2] = yy
        cv2.circle(img, (int(round(il)),int(round(it))), psr_r, (0,0,255), 1)
        cv2.imshow('convolution', img)
    return (dt, dl), peak, psr

In [7]:
def track(images, gt, l=.1, nu=0.01, exp=0.5, psr_r=4, score_thr=0.5, is_show=True):
    n_fails = 0
    score = 0.
    scales = [1.02, 1., 0.98]
    dtdl  = np.empty((len(scales), 2), dtype=int)
    peaks = np.empty((len(scales), 1), dtype=np.float)
    psrs = np.empty((len(scales), 1), dtype=np.float)
    bbox = gt[0]
    target = process_target(images[0], bbox, exp, True)
    ker, A, B = learn_mccf([target], y, l)
    
    for i in range(1, len(gt)):
        for j in range(len(scales)):
            dtdl[j], peaks[j], psrs[j] = new_pos(process_target(images[i], bbox, exp*scales[j], False), ker, psr_r, is_show)
        psrs[1]*=1.000001
        ind = psrs.argmax()
     
        (dtop, dleft), scale = dtdl[ind], scales[ind]
        
        bbox = correct_box(bbox, dtop, dleft, scale, exp)
        target = process_target(images[i], bbox, exp, True)
        ker, A, B = increment_mccf(A, B, [target], y, nu=nu, l=l)
        fscore, intersec = frame_score(gt[i], bbox)
        
        if is_show: show(images[i].copy(), bbox, gt[i], intersec)    
        if fscore < score_thr:
            n_fails += 1
            bbox = gt[i]
            break
        else:
            score += fscore
    return n_fails, score / len(gt)

In [24]:
fsize = 128
csize = 2
bsize = 2
cell_size = (csize, csize)  # h x w in pixels
block_size = (bsize, bsize)  # h x w in cells
nbins = 36  # number of orientation bins
hsize = fsize / csize


hog_desc = cv2.HOGDescriptor((fsize, fsize), (block_size[1] * cell_size[1], block_size[0] * cell_size[0]),
                        (cell_size[1], cell_size[0]), (cell_size[1], cell_size[0]), nbins)

sigma=4
y = gaussian_peak(hsize, var=[[sigma, 0], [0, sigma]])

win = np.hanning(fsize).reshape(1, -1); 
win = win.astype(np.float32)
win = win.T.dot(win)

In [25]:
obj_id = 0
gt, images = sequence(obj_id)

In [ ]:
track(images,
      gt,
      l=.1, #regularization
      nu=.01,#learning rate
      exp=.5, #expansion
      psr_r = 4,
      score_thr=0.3
     )